# Introduction au Big Data

Le Big Data fait référence a la collectionde données caractérisées par un `volume`, une `vélocité` et une `variété` si grands que leur transformation en valeur utilisable requiert l’utilisation de technologies et de méthodes analytiques spécifiques.

* Volume - pas d'échantillonnage, on observe et mesure tout
* Vélocité - les données et les résultats sont souvent disponibles en temps réel
* Variété - puise dans les données textuelles, les photos, audio / vidéo et complète généralement les pièces manquantes en fusionnant plusieurs sources

![bigdata.png](images/bigdata.png)

## Pourquoi le Big Data ?

*“There was 5 exabytes of information created between the dawn of civilization through 2003, but that much information is now created every 2 days, and the pace is increasing.”*

-- Eric Schmidt, PDG Google, 2010

* Augmentation exponentielle de la quantité de données non structurées 
    - Email, chat, blog, web, musique, photo, vidéo, etc.
* Augmentation de la capacité de stockage et d’analyse 
    - L’utilisation de plusieurs machines en parallèle devient accessible
* Les technologies existantes ne sont pas conçues pour ingérer ces données 
    - Base de données relationnelles, tableurs (Excel), etc.
* De “nouvelles” technologies et techniques d’analyse sont nécessaires
    - “Google File System” - Google 2003
    - “MapReduce: Simplified Data Processing on Large Clusters” - Google, 2004
    - Hadoop: circa 2006
* D’où le “Big Data”: pas strictement plus de data... 

## Les applications

![bigdata-applications.png](images/bigdata-applications.png)

## Les techniques et les technologies

### Systemes de fichiers distribués

La base du “Big Data”: le stockage
* Volume: une énorme quantité de stockage (à un coût raisonnable)
* Vélocité: agrandir la capacité de manière progressive
* Variété: un système de fichier “général”, qui permet de stocker n’importe quel genre de donnée

* Systèmes traditionnels: SAN (Storage Area Network)
* Coût initial (très) élevé
* Peut atteindre une très grande capacité, mais éventuellement limitée
    -Installer ou migrer un SAN est très coûteux (temps + $)
* Généralement sur du matériel et/ou OS propriétaire qui ne permet pas de lancer ses propres tâches
* Système “scale-up” plutôt que “scale-out”

![scale_up_vs_out.png](images/scale_up_vs_out.png)

### Hadoop Distributed File System (HDFS)

* Du besoin est né “Google File System” qui a inspiré la création de Hadoop / HDFS
* Hadoop Distributed File System est composé de 2 services:
     - Namenode: service de méta-données
     - Datanode: service de bloc de données
* Un fichier inscrit dans HDFS sera divisé en blocs
     - généralement de 64MB chacun
* Les blocs sont répliqués (généralement 3x) et distribués sur plusieurs Datanode
* Les clients du système de fichiers interrogent le Namenode pour:
     - connaître la structure de l’arbre de fichiers
     - découvrir où se trouvent les blocs d’un fichier
* Les clients accèdent aux données directement auprès des Datanode

![hdfs_archi.png](images/hdfs_archi.png)

* Les fichiers sont divisés en blocs: la perte d’un seul de ces blocs causerait une corruption du fichier
* Les blocs sont répliqués afin de survivre à la perte d’un ou plusieurs blocs
* Le Namenode tente de placer les blocs afin d’éviter la perte de toutes les copies d’un même bloc d’un seul coup (défaillance d’un “rack” ou d’une “switch”)
     - “Rack-awareness”
* Les blocs peuvent être placés n’importe où, le client doit interroger le Namenode pour trouver un bloc
* Un Namenode “stand-by” peut être déployé
     - Nécessite d’autres services: Journalnode
     - Basculement automatique: Zookeeper et “zkfc”

* Avantages:
     - En production dans des milliers de compagnies (“battle-hardened”)
     - Documentation
     - Compatibilité - pratiquement tout l’écosystème “Big Data” parle HDFS
     - Disponibilité du support (les “vendeurs” Hadoop)
* Désavantages:
     - La configuration “HA” est complexe et fragile
     - Ne supporte que la réplication pour éviter la perte de données
     - Aucune capacité de fédération (grappes de HDFS)
     - La nécessité d’interroger le Namenode pour chaque lecture
     - Nombre de blocs limité par l’espace mémoire du Namenode (Scale-up du Namenode)
     - Relativement lent

Il existe d'autres systemes de fichiers distribues mais leur etude sort du scope de ce cours. Parmi eux on peut citer:
* Cloud Storage de Google
* S3 d'Amazon
* CEPH de RedHat

## Algorithmes distribués

* Historiquement dominés par des systèmes haute-performance
    - problèmes “cpu-bound”, peu de données, calculs complexes
    - p. ex: MPI (message passing interface), les données sont envoyées aux agents, le calcul effectué et les résultats retournés au “coordonnateur”
* Afin de traiter de très grandes quantités de données, inverser la responsabilité: déplacer l’algorithme vers les données
    - “data-locality”
- Afin de maximiser le parallélisme: réduire la dépendance entre les “agents”
- Formalisation: map / reduce

### MapReduce (MR)

* Algorithme composé de 2 étapes conceptuelles
     - map - transformation des données en paires de clé-valeur
     - reduce - opération d'agrégation (somme, moyenne, etc.) par clé
* L’implémentation nécessite plus de 2 étapes, mais elles sont généralement transparentes pour le programmeur
* Sa résilience et son parallélisme sont ce qui le rendent particulièrement intéressant pour le Big-Data
     - “Embarassingly parallel”
     - Capacité de redémarrer n’importe quelle sous-étape (tant que les données source existent)
 

![map_reduce.png](images/map_reduce.png)

![map_reduce_exple.png](images/map_reduce_exple.png)

* À moins d’avoir un problème très simple, une seule phase M/R n’est pas suffisante
* Généralement, on doit écrire plusieurs phases et les exécuter en chaîne
* Manuellement:
    - laborieux
    - sujet à l’erreur
    - possibilités d’optimisations potentiellement perdues
* Plusieurs abstractions ont été créées
    - Certaines ont des optimisations intéressantes (map-side aggregation)
    - Écrire le Map/Reduce directement n’est pas tellement conseillé aujourd’hui

* Hive
    - “compile” le SQL en chaîne de phases M/R
    - permet la gestion de données structurées (tabulaires)
    - s’adresse principalement aux analystes d’affaires (BI), création de rapports, etc.
- Pig
    - langage haut-niveau (quelques emprunts à SQL) compilé en phases M/R
    - s’adresse principalement aux programmeurs / chercheurs
* Cascading
    - abstraction de “pipes-and-filters” sur Hadoop
    - contient un optimiseur d’exécution (plan logique vs. plan physique)
    - interface java, scala, clojure, ruby, python et SQL
    - s’adresse aux programmeurs
* et beaucoup d’autres...

### Spark

* Ré-implémentation d’une plateforme de calcul distribué
    - contient les mêmes étapes conceptuelles que Map/Reduce
    - n’utilise pas Map/Reduce de Hadoop
    - plan d’exécution plus sophistiqué
    - p. ex.: ne nécessite pas l’écriture sur disque à chaque étape
* Unité opérationnelle est le RDD: resilient distributed dataset

![rdd.png](images/rdd.png)

![rdd_lineage.png](images/rdd_lineage.png)

* Spark est une plateforme: une même abstraction pour différents cas d’usage
    - spark: tâches “batch”
    - spark-streaming: tâches en flux continu de données
    - spark-mllib: apprentissage machine
    - spark-graphx: manipulation de graph
    - spark-sql: abstraction SQL
    - spark-R: exécution de R sur spark
    - pyspark: exécution de python sur spark
    - des dizaines d’extensions...

### Streaming vs. batch

* Initialement, le big-data opérait en “batch”
    - création d’index, rapports, etc. à chaque jour (24 h)
    - pression d’obtenir des résultats de plus en plus rapidement
* Problème fondamental: durée d’une tâche batch doit être < interval entre les résultats
* Nécessité de pouvoir calculer “incrémentalement” en flux continu
    - p. ex.: opérer sur chaque “tweet”
* Différentes plateformes sont créées:
    - Storm (Twitter), Samza (LinkedIn), Spark Streaming

#### Spark streaming

![spark_streaming.png](images/spark_streaming.png)

#### Dstream transformations

![dstream_trans.png](images/dstream_trans.png)

* Initialement, les tâches batches sont distinctes des tâches en flux 
    - duplication de “logique d’affaire”
* Les tâches en flux calculent incrémentalement (possiblement de manière inexacte)
* Les tâches batches sont utilisées pour “réparer”, s’assurer de l’exactitude des données
    - ou simplement complémenter
* C’est le “lambda architecture”
    - Architecture qui utilise des processus batch et en flux
* Spark permet aujourd’hui d’unifier les 2 mondes
    - Le mode d’opération d’une tâche est un détail de déploiement

### Systèmes de base de donnees

* Technologies - systèmes de base de données
* Les paradigmes de batch et de flux sont insuffisants
    - Il est aussi nécessaire de lire et écrire aléatoirement (random read/write)
* Les bases de données traditionnelles (du type scale-up) ne sont pas appropriées
    - Volume: un seul serveur ne peut plus contenir toutes les données
    - Vélocité: la bande passante d’un seul serveur ne peut pas soutenir le taux de requêtes
    - Variété: les données ne sont pas toutes tabulaires (relationnelles)
* C’est la naissance du “NoSQL”
    - un pauvre choix de nom
    - ne décrit pas ce que le système est, mais plutôt ce qu’il n’est pas
    - plusieurs BD “NoSQL” ont une interface SQL (ou simili-SQL)

#### NoSQL

* Généralement (et non strictement), une BD NoSQL:
    - offre un paradigme d’accès ou stockage non relationnel
    - est distribuée
    - offre une certaine forme de capacité “scale-out”
    - utilise un design simple (qui offre parfois peu de fonctionnalités)
    - utilise une architecture sans point de défaillance unique
* Étant donné le design simple, une BD NoSQL peut:
    - soutenir un taux de requête très grand
    - survivre à des défaillances réseau ou de noeud
    - offrir une capacité très grande de stockage

* Le NoSQL n’est pas magique: on échange les garanties des BD traditionnelles contre ces avantages:
    - pas de transaction (begin, commit / rollback)
    - perte des garanties de durabilité (perte d’une écriture confirmée)
    - face aux défaillances réseau, doit choisir entre cohérence et disponibilité
* De nombreux autres problèmes:
    - projets immatures: nombreux bugs, perte de données en production (même sans défaillance)
    - défaillance architecturales fondamentales: algorithmes de consensus brisé
    - installation et/ou opérations parfois très complexe
    - très peu, voire aucun support pour les entreprises (nécessite expertise interne)
* Ce sont, malgré tout, des outils indispensables
    - généralement, le choix se limite à une ou deux technologies
    - le choix est basé sur les garanties fournies ainsi que la “famille”

* Les familles (selon le paradigme d’accès): 
    - Document (p. ex. MongoDB, Elasticsearch)
    - Clé-valeur / famille de colonnes (p.ex. HBase, Cassandra)
    - Graphe
    - Structuré / semi-structuré
* Non exhaustif, c’est une catégorisation parmi d’autres

## Conclusion

* Les 3 Vs du Big Data ont poussé les limites des systèmes traditionnels
* Nécessaire d’opérer sur plusieurs noeuds en parallèle
* De nouvelles techniques et technologies ont vu le jour
    - systèmes de fichiers distribués
    - algorithmes distribués
    - systèmes de base de données distribués
* Les systèmes distribués sont complexes
    - doivent faire des compromis
    - ces compromis dictent les caractéristiques fondamentales de ces systèmes
    - important de bien les connaître
* De nouvelles plateformes émergent pour simplifier la gestion de ces systèmes

## References

[ETS 2017.07 - Philippe Laflamme](https://cours.etsmtl.ca/log660/public_docs/acetates/BigData_Technologies_PL.pdf)